In [1]:
import os
import numpy as np
import pandas as pd
import swifter
import utils
import catboost

In [2]:
DATA_PATH = "./data"

In [3]:
test = utils.load_full_test_csv(DATA_PATH)

In [4]:
closest_hits_features = test.swifter.apply(
    utils.find_closest_hit_per_station, result_type="expand", axis=1)
test_concat = pd.concat(
    [test.loc[:, utils.SIMPLE_FEATURE_COLUMNS], closest_hits_features],
    axis=1)

Pandas Apply: 100%|██████████| 726095/726095 [09:44<00:00, 1241.54it/s]


In [5]:
model = catboost.CatBoostClassifier().load_model("track_2_model.cbm")
predictions = model.predict(test_concat.values, prediction_type="RawFormulaVal").astype(np.float32)
prediction_pd = pd.DataFrame(data={"prediction": predictions}, index=test.index)

Check that C++ and python produce the same predictions. To produce the referred files, run:
```
cd track_2_baseline
make baseline compute_features
./baseline > submission_cpp.csv < test_public.csv
./compute_features > features_cpp.csv < test_public.csv
```

In [6]:
features_cpp = np.genfromtxt("track_2_baseline/features_cpp.csv", delimiter=" ")

In [7]:
np.testing.assert_allclose(features_cpp, test_concat.values)

In [12]:
predictions_cpp = pd.read_csv("track_2_baseline/submission_cpp.csv", index_col=utils.ID_COLUMN,
                              dtype={"prediction": np.float32, utils.ID_COLUMN: np.int64})

In [13]:
np.testing.assert_allclose(prediction_pd, predictions_cpp)